<div class="alert alert-block alert-success">
<h1>Numerical Integration</h1>
</div>

<p style="font-size:17px; color:black; text-align:justify">The basic problem in numerical integration is to compute an approximate solution to a definite integral to a given degree of accuracy. The simplest method of integration is to let the interpolating function be a constant function (a polynomial of degree zero) that passes through a point. This is called the midpoint rule or rectangle rule.</p>

<p style="font-size:17px; color:black; text-align:justify">Below is an example of the rectangle rule, implemented on function: $f(x)=\frac{1}{x} \sin (x) + 0.5$</p> 

<p style="font-size:17px; color:red; text-align:justify">The plot is interactive, change the step size to see its effect on the accuracy.</p> 

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [29]:
# HIDDEN
# General Purpose
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy.integrate import odeint
import matplotlib.ticker as ticker
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.cm as cm
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from sympy import *


# Jupyter Specifics
from IPython.display import HTML
#from ipywidgets.widgets import interact, IntSlider, FloatSlider, Layout

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider, FloatSlider, Layout


%matplotlib inline
matplotlib.rcParams['xtick.direction'] = 'out'
matplotlib.rcParams['ytick.direction'] = 'out'

style = {'description_width': '150px'}
slider_layout = Layout(width='90%')

In [68]:
# HIDDEN
c = 1  # chord length

lowerPressure = '2*10**4*(x-1)**2 + 1.73*10**5'   # lower level pressure
upperPressure = '4*10**4*(x-1)**2 + 5.4*10**4'  # upper level pressure
lowerShear = '731*x**(-0.2)' # lower level shear stress
upperShear = '288*x**(-0.2)' # upper level shear stress


def Pl(x):
    return eval(lowerPressure)


def Pu(x):
    return eval(upperPressure)


def Tl(x):
    return eval(lowerShear)


def Tu(x):
    return eval(upperShear)


x = Symbol('x')
integrtionRange = (x, 0, c)


# Normal force
def N():
    return float(integrate(lowerPressure + "-(" + upperPressure + ")", integrtionRange))


# Axial force
def A():
    return float(integrate(lowerShear + "+(" + upperShear + ")", integrtionRange))


# Lift
def L(a):
    a = a * np.pi / 180  # convert angle to degrees
    Ap = A()
    Np = N()
    return float(Np*np.cos(a) - Ap*np.sin(a))


# Lift coefficient
def Cl(a, ca,cn):
    a = a * np.pi / 180  # convert angle to degree
    return float(cn*np.cos(a) - ca*np.sin(a))


# Drag
def D(a):
    a = a * np.pi / 180  # convert angle to degrees
    Ap = A()
    Np = N()
    return float(Ap*np.cos(a) + Np*np.sin(a))

# drag coefficient
def Cd(a, ca, cn):
    a = a * np.pi / 180  # convert angle to degrees
    return float(ca*np.cos(a) + cn*np.sin(a))
 
    
# Moment at the leading edge
def Mle():
    return float(integrate("(" + upperPressure + "-(" + lowerPressure + "))*x", integrtionRange))


# Moment at the quarter chord
def Mc4(a):
    a = a * np.pi / 180  # convert angle to degrees
    return float(Mle() + L(a)*(c/4))


# Locaion of center of pressure
def Xcp(a):
    a = a * np.pi / 180  # convert angle to degrees
    return float(-Mle()/N())

In [84]:
from IPython.display import display

alphaDefault = 12
cnDefault = 1.2
caDefault = 0.03

style = {'description_width': 'initial'}
sliderLayout = Layout(width='80%')
boxLayout = Layout(width='70%')

# Angle of attack widget settings
alpha_widget = widgets.FloatSlider(
    value=alphaDefault,
    min=0,
    max=15.0,
    step=0.5,
    description=r'Angle of Attack ($^{\circ}$):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=sliderLayout,
    style=style
)
display(alpha_widget)


# Axial force coeefficient widget settings
ca_widget = widgets.FloatSlider(
    value=caDefault,
    min=-1,
    max=1,
    step=0.01,
    description=r'Axial Force Coefficient, ($c_a$):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=sliderLayout,
    style=style
)


# Normal force coeefficient widget settings
cn_widget = widgets.FloatSlider(
    value=cnDefault,
    min=-3,
    max=3,
    step=0.1,
    description=r'Norma Force Coefficient ($c_n$):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=sliderLayout,
    style=style
)






# Lift coeficient widget settings
LiftCE = widgets.FloatText(
    value=Cl(alphaDefault, caDefault, cnDefault),
    description=r"Lift Coefficient ($N$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
    layout=boxLayout,
    style=style
)

def update_liftce(*args):
    LiftCE.value = Cl(alpha_widget.value, ca_widget.value, cn_widget.value)
    
ca_widget.observe(update_liftce, 'value')
alpha_widget.observe(update_liftce, 'value')
cn_widget.observe(update_liftce, 'value')


# Drag coeficient widget settings
DragCE = widgets.FloatText(
    value=Cd(alphaDefault, caDefault, cnDefault),
    description=r"Drag Coefficient ($N$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
    layout=boxLayout,
    style=style
)

def update_dragce(*args):
    DragCE.value = Cd(alpha_widget.value, ca_widget.value, cn_widget.value)
    
ca_widget.observe(update_dragce, 'value')
alpha_widget.observe(update_dragce, 'value')
cn_widget.observe(update_dragce, 'value')


display(ca_widget)
display(cn_widget)
display(LiftCE)
display(DragCE)


# Normal Force widget settings
NormalForce = widgets.FloatText(
    value=N(),
    description=r"Normal Force ($N$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_normal(*args):
    NormalForce.value = N()
alpha_widget.observe(update_normal, 'value')
display(NormalForce)


# Axial Force widget settings
AxialForce = widgets.FloatText(
    value=A(),
    description=r"Axial Force ($N$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_axial(*args):
    AxialForce.value = A()
alpha_widget.observe(update_axial, 'value')
display(AxialForce)



# Lift widget settings
Lift = widgets.FloatText(
    value=L(alphaDefault),
    description=r"Lift ($N$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_lift(*args):
    Lift.value = L(alpha_widget.value)
alpha_widget.observe(update_lift, 'value')
display(Lift)



# Drag widget settings
Drag = widgets.FloatText(
    value=D(alphaDefault),
    description=r"Drag ($N$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_drag(*args):
    Drag.value = D(alpha_widget.value)
alpha_widget.observe(update_drag, 'value')
display(Drag)




# Moment at the leading edge widget settings
MomLE = widgets.FloatText(
    value=Mle(),
    description=r"Moment at the Leading Edge ($N\cdot m$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_mle(*args):
    MomLE.value = Mle()
alpha_widget.observe(update_mle, 'value')
display(MomLE)



# Moment at the Quarter Chord widget settings
MomC4 = widgets.FloatText(
    value=Mc4(alphaDefault),
    description=r"Moment at the Quarter Chord ($N\cdot m$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_mc4(*args):
    MomC4.value = Mc4(alpha_widget.value)
alpha_widget.observe(update_mc4, 'value')
display(MomC4)



# Location of Center of Pressure widget settings
LocCoP = widgets.FloatText(
    value=Xcp(alphaDefault),
    description=r"Center of Pressure Location ($m$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=boxLayout,
    style=style
)
def update_xcop(*args):
    LocCoP.value = Xcp(alpha_widget.value)
alpha_widget.observe(update_xcop, 'value')
display(LocCoP)






FloatSlider(value=12.0, continuous_update=False, description='Angle of Attack ($^{\\circ}$):', layout=Layout(w…

FloatSlider(value=0.03, continuous_update=False, description='Axial Force Coefficient, ($c_a$):', layout=Layou…

FloatSlider(value=1.2, continuous_update=False, description='Norma Force Coefficient ($c_n$):', layout=Layout(…

FloatText(value=1.167539770156034, description='Lift Coefficient ($N$):', layout=Layout(width='70%'), style=De…

FloatText(value=0.2788384570033253, description='Drag Coefficient ($N$):', layout=Layout(width='70%'), style=D…

...

FloatText(value=109613.7529662517, description='Lift ($N$):', layout=Layout(width='70%'), style=DescriptionSty…

FloatText(value=24601.32877496298, description='Drag ($N$):', layout=Layout(width='70%'), style=DescriptionSty…

FloatText(value=-57833.333333333336, description='Moment at the Leading Edge ($N\\cdot m$):', layout=Layout(wi…

FloatText(value=-29751.35164147804, description='Moment at the Quarter Chord ($N\\cdot m$):', layout=Layout(wi…

FloatText(value=0.5148367952522256, description='Center of Pressure Location ($m$):', layout=Layout(width='70%…

In [80]:
alphaDefault = 12
cnDefault = 1.2
caDefault = 0.03


# Angle of attack widget settings
alpha_widget = widgets.FloatSlider(
    value=alphaDefault,
    min=0,
    max=15.0,
    step=0.5,
    description=r'Angle of Attack ($^{\circ}$):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
    layout=sliderLayout,
    style=style
)



# Axial force coeefficient widget settings
ca_widget = widgets.FloatSlider(
    value=caDefault,
    min=-1,
    max=1,
    step=0.01,
    description=r'Axial Force Coefficient, ($c_a$):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=sliderLayout,
    style=style
)


# Normal force coeefficient widget settings
cn_widget = widgets.FloatSlider(
    value=cnDefault,
    min=-3,
    max=3,
    step=0.1,
    description=r'Norma Force Coefficient ($c_n$):',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
    layout=sliderLayout,
    style=style
)






# Lift coeficient widget settings
LiftCE = widgets.FloatText(
    value=Cl(alphaDefault, caDefault, cnDefault),
    description=r"Lift Coefficient ($N$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
    layout=boxLayout,
    style=style
)

def update_liftce(*args):
    LiftCE.value = Cl(alpha_widget.value, ca_widget.value, cn_widget.value)
    
ca_widget.observe(update_liftce, 'value')
alpha_widget.observe(update_liftce, 'value')
cn_widget.observe(update_liftce, 'value')


# Lift coeficient widget settings
DragCE = widgets.FloatText(
    value=Cd(alphaDefault, caDefault, cnDefault),
    description=r"Drag Coefficient ($N$):",
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.5f',
    layout=boxLayout,
    style=style
)

def update_dragce(*args):
    DragCE.value = Cd(alpha_widget.value, ca_widget.value, cn_widget.value)
    
ca_widget.observe(update_dragce, 'value')
alpha_widget.observe(update_dragce, 'value')
cn_widget.observe(update_dragce, 'value')

display(alpha_widget)
display(ca_widget)
display(cn_widget)
display(LiftCE)
display(DragCE)


FloatSlider(value=12.0, continuous_update=False, description='Angle of Attack ($^{\\circ}$):', layout=Layout(w…

FloatSlider(value=0.03, continuous_update=False, description='Axial Force Coefficient, ($c_a$):', layout=Layou…

FloatSlider(value=1.2, continuous_update=False, description='Norma Force Coefficient ($c_n$):', layout=Layout(…

FloatText(value=1.167539770156034, description='Lift Coefficient ($N$):', layout=Layout(width='70%'), style=De…

FloatText(value=0.2788384570033253, description='Drag Coefficient ($N$):', layout=Layout(width='70%'), style=D…

In [44]:
import ipywidgets as widgets
from IPython.display import display

geo={'USA':['CHI','NYC'],'Russia':['MOW','LED']}
geoWs = {key: widgets.Select(options=geo[key]) for key in geo}

def get_current_state():
    return {'country': i.children[0].value,
            'city': i.children[1].value}

def print_city(**func_kwargs):
    print('func_kwargs', func_kwargs)
    print('i.kwargs', i.kwargs)
    print('get_current_state', get_current_state())

def select_country(country):
    new_i = widgets.interactive(print_city, country=countryW, city=geoWs[country['new']])
    i.children = new_i.children

countryW = widgets.Select(options=list(geo.keys()))
init = countryW.value
cityW = geoWs[init]

countryW.observe(select_country, 'value')

i = widgets.interactive(print_city, country=countryW, city=cityW)

display(i)

interactive(children=(Select(description='country', options=('USA', 'Russia'), value='USA'), Select(descriptio…

In [6]:
import ipywidgets as widgets


x_widget = FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = widgets.FloatText()

def update_y_range(*args):
    y_widget.value = 2.0 * x_widget.value
x_widget.observe(update_y_range, 'value')

display(x_widget)
display(y_widget)





FloatSlider(value=0.0, max=10.0, step=0.05)

FloatText(value=0.0)

In [7]:
# HIDDEN
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon, Rectangle


def func2(x):
    return (x - 3) * (x - 5) * (x - 7) + 300

def func(x):
    return np.sin(x)/x + 0.5# y = sin(x)/x


x = np.linspace(-20,20,500)
min_x = min(x)
max_x = max(x)

def areaUnderCurve(a, dx):
    
    b = a + dx
    y = func(x)
    

    fig, ax = fig, ax = plt.subplots(figsize=(15, 8))
    ax.plot(x, y, 'r', linewidth=2, label='(x-3)(x-5)(x-9)+85')
    ax.set_ylim(bottom=0)

    # Make the shaded region
    ix = np.linspace(a, b)
    iy = func(0.5*(a+b))
    verts = [(a, 0), (a,iy), (b,iy), (b, 0)]
#     verts = [*zip(ix, iy)]
    poly = Polygon(verts, facecolor='0.75', edgecolor='0.1')
#     poly = Rectangle((ix, iy), dx,1,facecolor='0.75', edgecolor='0.1')
    ax.add_patch(poly)

    # ax.text(0.5 * (a + b), 30, r"$\int_a^b f(x)\mathrm{d}x$",
    #         horizontalalignment='center', fontsize=20)

    fig.text(0.9, 0.05, '$X$')
    fig.text(0.1, 0.9, '$Y$')

#     ax.spines['right'].set_visible(False)
#     ax.spines['top'].set_visible(False)
    ax.xaxis.set_ticks_position('bottom')

    ax.set_xticks((a, b))
    ax.set_xticklabels((r'a'))
    ax.set_yticks([])
    plt.show()

In [8]:
func2(4)

303

In [9]:
# HIDDEN
interactive_plot = interactive(areaUnderCurve, 
                               a=widgets.FloatSlider(value=4,min=min_x,max=max_x,step=0.25,description=r'$a$', style=style, layout=slider_layout), 
                               dx=widgets.FloatSlider(value=1.0,min=0.1,max=max_x,step=0.1,description=r'$dx$', style=style, layout=slider_layout));
   
#interactive_plot

In [10]:
# HIDDEN
def findArea(dx):
    a = min_x
    b = a + dx
    y = func(x)
    Area = 0

    fig, ax = fig, ax = plt.subplots(figsize=(15, 8))
    ax.plot(x, y, 'r', linewidth=2, label='(x-3)(x-5)(x-9)+85')
    ax.set_ylim(bottom=0)

    # Make the shaded region
    ix = np.linspace(a, b)
    
    
    range = max_x - min_x
    
    numOfPoly = range / dx
    
    
    for i in np.arange(0,numOfPoly):
        
        if (a+dx>max_x):
            dx = max_x - a
        iy = func(0.5*(a+b))
        verts = [(a, 0), (a,iy), (b,iy), (b, 0)]
#     verts = [*zip(ix, iy)]
        poly = Polygon(verts, facecolor='0.75', edgecolor='0.1')
#     poly = Rectangle((ix, iy), dx,1,facecolor='0.75', edgecolor='0.1')
        ax.add_patch(poly)
        a += dx
        b = a + dx
        Area += dx*iy

    # ax.text(0.5 * (a + b), 30, r"$\int_a^b f(x)\mathrm{d}x$",
    #         horizontalalignment='center', fontsize=20)

    fig.text(0.9, 0.05, '$X$')
    fig.text(0.1, 0.9, '$Y$')
    
    print("Predicted Area: ", Area, "  True Area: ", 23.096522312, "  Error: ", 100*abs(23.096522312-Area)/23.096522312, "%")

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.xaxis.set_ticks_position('bottom')

    ax.set_xticks((a, b))
    ax.set_xticklabels((r'a'))
    ax.set_yticks([])
    plt.show()

In [11]:
# HIDDEN
interactive_plot = interactive(findArea, 
                               dx=widgets.FloatSlider(value=1.0,min=0.1,max=8,step=0.1,description=r'$dx$', style=style, layout=slider_layout))
   
# areaUnderCurve(1,0.5)
interactive_plot

interactive(children=(FloatSlider(value=1.0, description='$dx$', layout=Layout(width='90%'), max=8.0, min=0.1,…

In [12]:
# HIDDEN

# some note on interact

def f(x):
    return x


interact(f, x=10);
interact(f, x=True);
interact(f, x='Hi there!');

@interact(x=True, y=1.0)
def g(x, y):
    return (x, y)


def h(p, q):
    return (p, q)

interact(h, p=5, q=fixed(20));

interact(f, x=widgets.IntSlider(min=-10, max=30, step=1, value=10));

interact(f, x=(0,8,2));


@interact(x=(0.0,20.0,0.5))
def h(x=5.5):
    return x

interact(f, x=['apples','oranges']);
interact(f, x=[('one', 10), ('two', 20)]);



a = widgets.IntSlider()
b = widgets.IntSlider()
c = widgets.IntSlider()
ui = widgets.HBox([a, b, c])
def f(a, b, c):
    print((a, b, c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)


x_widget = FloatSlider(min=0.0, max=10.0, step=0.05)
y_widget = FloatSlider(min=0.5, max=10.0, step=0.05, value=5.0)

def update_x_range(*args):
    x_widget.max = 2.0 * y_widget.value
y_widget.observe(update_x_range, 'value')

def printer(x, y):
    print(x, y)
interact(printer,x=x_widget, y=y_widget);

def f(m, b):
    plt.figure(2)
    x = np.linspace(-10, 10, num=1000)
    plt.plot(x, m * x + b)
    plt.ylim(-5, 5)
    plt.show()

interactive(f, m=(-2.0, 2.0), b=(-3, 3, 0.5))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot


def f5(x):
    return x



widgets.BoundedFloatText(
    value=1,
    min=0,
    max=10.0,
    step=0.1,
    description='Text:',
    disabled=False
)

interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

interactive(children=(Checkbox(value=True, description='x'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Text(value='Hi there!', description='x'), Output()), _dom_classes=('widget-interact',))

interactive(children=(Checkbox(value=True, description='x'), FloatSlider(value=1.0, description='y', max=3.0, …

interactive(children=(IntSlider(value=5, description='p', max=15, min=-5), Output()), _dom_classes=('widget-in…

...

interactive(children=(Dropdown(description='x', options=('apples', 'oranges'), value='apples'), Output()), _do…

interactive(children=(Dropdown(description='x', options=(('one', 10), ('two', 20)), value=10), Output()), _dom…

Output()

interactive(children=(FloatSlider(value=0.0, description='x', max=10.0, step=0.05), FloatSlider(value=5.0, des…

BoundedFloatText(value=1.0, description='Text:', max=10.0, step=0.1)